In [1]:
ls

Data Generation.ipynb  dumps/                 ptb.vocab.json
README.md              figs/                  requirements.txt
__pycache__/           inference.py           simple-examples/
bin/                   model.py               simple-examples.tgz
data/                  ptb.py                 train.py
data-copy/             ptb.train.txt          utils.py
dowloaddata.sh*        ptb.valid.txt


In [2]:
ls data

clean_poems.csv      ptb.train.json       ptb.vocab.json
final_poems.csv      ptb.train.txt        ultimate_poems.csv
glove.6B.300d.txt    ptb.valid.json
glove.840B.300d.txt  ptb.valid.txt


In [3]:
import pandas as pd
import json
import spacy

In [4]:
with open('data-copy/ptb.vocab.json') as data_file:    
    data = json.load(data_file)
    
print(len(data['w2i']))
print(len(data['i2w']))
#print(data)
for k, v in data.items():
    print(k)

9877
9877
w2i
i2w


In [5]:
import torch
# prepare glove vectors
words = set()
word_to_vec = {}
word_to_id = {}
id_to_word = {}
glove_dim = 300
glove_filename = "data/glove.840B.300d.txt" # "./glove/glove.6B.300d.txt"
with open(glove_filename) as file:
    count = 0
    for line in file:
        tokens = line.split()
        word, vals = tokens[0], tokens[-glove_dim:]
        if word not in words:
            words.add(word)
            #word_to_vec[word] = torch.Tensor([float(val) for val in vals])
            word_to_id[word] = count
            id_to_word[count] = word
            count += 1
print(len(words))
print(id_to_word[1])

2195884
.


In [6]:
def make_embedding(words, id_to_word, word_to_vec, dim):
    weights = torch.stack([word_to_vec[id_to_word[i]] for i in range(len(words))])
    embed = nn.Embedding.from_pretrained(weights)
    return embed

In [7]:
def load_data(vocab, poems, limit):
    nlp = spacy.load('en')
    line_data = []
    scraps = {}
    count = 0
    for poem in poems:
        try:
            for line in poem.lower().split('\n'):
                if len(line) == 0:
                    continue
                tokens = nlp(line)
                include = True
                bad_words = []
                for token in tokens:
                    if token.text not in vocab:
                        bad_words.append(token.text)
                        include = False
                if include: line_data.append(line)
                else: scraps[line] = bad_words
        except:
            continue
        if count == limit:
            break
        count += 1
    return line_data, scraps

In [8]:
def prepare_training_data(limit=-1):
    filename = "./data/ultimate_poems.csv"
    data = pd.read_csv(filename)
    #poems = data.iloc[:,3]
    poems = load_data(words, data, limit)
    return poems
    
def sentence_to_vector(s, word_to_vec):
    ids = [word_to_vec[w] for w in s]
    return torch.tensor(ids, dtype=torch.long)

def sentence_one_hot(s, word_to_vec):
    one_hot = torch.zeros([len(s), len(word_to_vec)], dtype=torch.long)
    for i, w in enumerate(s):
        one_hot[i, word_to_vec[w]] = 1
    return one_hot

train_data, scraps = prepare_training_data(limit=-1)
print(len(train_data), len(scraps))
print(scraps)

15 1
{'and discrete-a mirror come unsilvered,': ['unsilvered']}


In [9]:
nlp = spacy.load('en')
filename = "data/ultimate_poems.csv"
poems = pd.read_csv(filename)

In [10]:
good_poems = []
bad_poems  = []

def valid_line(line, words):
    tokens = nlp(line)
    count = 0
    for token in tokens:
        w = token.text
        if w.isalnum():
            if w not in words:
                return False
            count += 1
    if count < 2:
        return False
    return True
            
for poem in poems.iloc[:,1]:
    if valid_line(poem, words): good_poems.append(poem)
    else: bad_poems.append(poem)
print(len(good_poems))
print(len(bad_poems))

8722
4155


In [11]:
lower_poems = [poem.lower() for poem in good_poems]
df = pd.DataFrame({'poems':lower_poems})
print(df.head())
df.to_csv(path_or_buf="./data/final_poems.csv")

                                               poems
0  we'd  like  to  talk  with  you  about  fear t...
1  the wise men will unlearn your name.\nabove yo...
2  winter\nmore time is spent at the window.\n\ns...
3  may i never be afraid\nespecially of myself\nb...
4  the shift of sleepwalks and suicides.\nthe occ...


In [2]:
ls data

clean_poems.csv      glove.840B.300d.txt  ptb.vocab.json
final_poems.csv      ptb.train.txt        ultimate_poems.csv
glove.6B.300d.txt    ptb.valid.txt


In [13]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/johnhallman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
from nltk.tokenize import sent_tokenize, word_tokenize
poem_tokens = [word_tokenize(poem) for poem in lower_poems]

In [15]:
ls data-copy/

ptb.test.txt    ptb.train.txt   ptb.valid.txt
ptb.train.json  ptb.valid.json  ptb.vocab.json


In [16]:
vocab = set()
for tokens in poem_tokens:
    for token in tokens:
        vocab.add(token)
print(len(vocab))

54838


In [17]:
vocab_count = {}
for tokens in poem_tokens:
    for token in tokens:
        if token not in vocab_count:
            vocab_count[token] = 1
        else:
            vocab_count[token] += 1
print(len(vocab_count))
randcount = set()
for k, v in vocab_count.items():
    if v == 1:
        randcount.add(k)
print(len(randcount))

54838
26428


In [18]:
temp_poem_tokens = []
for tokens in poem_tokens:
    temp_poem_tokens.append(['<unk>' if token in randcount else token for token in tokens])

def ascii_word(w):
    for c in w:
        if ord(c) >= 128:
            return False
    return True

new_poem_tokens = []
for tokens in temp_poem_tokens:
    temp = []
    for token in tokens:
        if ascii_word(token): temp.append(token)
        else: temp.append('<unk>')
    new_poem_tokens.append(temp)

In [19]:
print(new_poem_tokens[4])

['the', 'shift', 'of', '<unk>', 'and', 'suicides', '.', 'the', 'occasion', 'of', 'owls', 'and', 'a', '<unk>', 'fog', '.', 'even', 'god', 'has', 'nodded', 'off', 'and', 'wo', "n't", 'be', 'taking', 'prayers', 'til', 'ten', '.', 'ad', 'interim', ',', 'you', 'put', 'them', 'on', '.', 'as', 'if', 'your', 'wants', 'could', 'keep', 'you', 'warm', '.', 'as', 'if', '.', 'you', 'say', 'your', '<unk>', '.', 'you', 'thumb', 'your', 'beads', '.', 'you', '<unk>', 'the', 'glass', '.', 'night', 'creeps', 'to', 'its', 'precipice', 'and', 'the', 'broken', 'rim', 'of', 'reason', 'breaks', 'again', '.', 'an', 'obsidian', 'sky', 'betrays', 'you', '.', 'every', '<unk>', 'shadow', '<unk>', 'you', '.', 'soon', 'enough', ',', 'the', 'crow', 'will', 'caw', '.', 'the', 'cock', 'will', 'crow', '.', 'the', 'door', 'will', 'close', '.', '(', 'he', 'is', "n't", 'coming', 'back', ',', 'you', 'know', '.', ')', 'and', 'so', 'wee', ',', 'wet', 'hours', 'of', 'grief', 'relent', '.', 'in', 'thirty', 'years', 'you', 'migh

In [20]:
def list_to_line(l):
    output = ""
    for w in l:
        output += w + " "
    return output

print(list_to_line(new_poem_tokens[1]))

the wise men will unlearn your name . above your head no star will flame . one weary sound will be the same- the hoarse roar of the gale . the shadows fall from your tired eyes as your lone bedside candle dies , for here the calendar breeds nights till stores of candles fail . what prompts this melancholy key ? a long familiar melody . it sounds again . so let it be . let it sound from this night . let it sound in my hour of death- as gratefulness of eyes and lips for that which sometimes makes us lift our gaze to the far sky . you glare in silence at the wall . your stocking gapes : no gifts at all . it 's clear that you are now too old to trust in good saint nick ; that it 's too late for miracles . -but suddenly , lifting your eyes to heaven 's light , you realize : your life is a sheer gift . 


In [21]:
print(len(new_poem_tokens))
train = new_poem_tokens[:8000]
valid = new_poem_tokens[8000:]
print(len(valid))

8722
722


In [22]:
with open("ptb.train.txt", "w") as file:
    for s in train:
        file.write(list_to_line(s))
        file.write('\n')
with open("ptb.valid.txt", "w") as file:
    for s in valid:
        file.write(list_to_line(s))
        file.write('\n')

In [23]:
count = 0
w2i = {}
i2w = {}
previous = set()
special = ["<pad>", "<unk>", "<sos>", "<eos>"]
for token in special:
    w2i[token] = count
    i2w[count] = token
    count += 1
for tokens in poem_tokens:
    for token in tokens:
        if token not in previous:
            w2i[token] = count
            i2w[count] = token
            previous.add(token)
            count += 1
final_vocab = {'w2i':w2i, 'i2w':i2w}

In [24]:
with open('ptb.vocab.json', 'w') as file:
    json.dump(final_vocab, file)

In [15]:
ls data

clean_poems.csv      glove.840B.300d.txt  ptb.vocab.json
final_poems.csv      ptb.train.txt        ultimate_poems.csv
glove.6B.300d.txt    ptb.valid.txt


In [6]:
test = "This is\na test"
from nltk.tokenize import sent_tokenize, word_tokenize
print(word_tokenize(test))

['This', 'is', 'a', 'test']


In [13]:
import pandas as pd
df = pd.read_csv("./data/final_poems.csv")

In [17]:
from nltk.tokenize import sent_tokenize, word_tokenize
final_lines = []
for poem in df.iloc[:,1]:
    lines = poem.split('\n')
    for line in lines:
        tokens = word_tokenize(line)
        if len(tokens) >= 2:
            final_lines.append(tokens)

In [18]:
vocab = set()
for line in final_lines:
    for token in line:
        vocab.add(token)
print(len(vocab))

54107


In [19]:
print(len(final_lines))

201824
